In [2]:
import pandas as pd
import numpy as np
import csv
import os
from datetime import datetime

In [3]:
path = '/Volumes/Cisco/Fall2021/onnx-exchange/miss-classification/'
path_output = '/Volumes/Cisco/Summer2022/onnx-exchange/analysis/conversion/'

In [4]:
models = ['lenet5', 'resnet18', 'vgg', 'lstm', 'gru']
frameworks = ['keras', 'pytorch']

In [6]:
data_file = open(path_output+'metrics_miss-classifications2.csv', mode='w', newline='', encoding='utf-8')
data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer.writerow(['model','Epoch', 'Training_accuracy', 'Validation_Accuracy', "Training_Loss", 'Validation_Loss'])
data_writer.writerow(['framework','model', 'loadtime_before', 'loadtime_onnx', 'loadtime_coreml', 'miss_classification_onnx', 'miss_classification_coreml', 'inferencetime_before', 'inferencetime_onnx', 'inferencetime_coreml', '', 'loadtime_before_std', 'loadtime_onnx_std', 'loadtime_coreml_std', 'miss_classification_onnx_std', 'miss_classification_coreml_std', 'inferencetime_before_std', 'inferencetime_onnx_std', 'inferencetime_coreml_std', 'miss_test_onnx', 'miss_test_coreml'])

for frame_ in frameworks:
    for model in models: 
        data_original_load_time = []
        data_original_infererence_time = []
        data_runtime_conversion_time = {}
        data_runtime_load_time = {}
        data_runtime_inference_time = {}
        data_encoded_miss_classified_original_test_runtime_percentage = {}
        data_encoded_miss_classified_original_runtime_percentage = {}

        if os.path.exists(path+'errors/{}/{}'.format(frame_,model)):
            list_files = [x for x in os.listdir(path+'errors/{}/{}'.format(frame_,model)) if 'runtime_miss-classification' in x]
            #print('yes', list_files)
            for i in range(1, 11):
                for file_ in list_files:
                    df = pd.read_csv(path+'errors/{}/{}/{}'.format(frame_,model, file_))
                    #print(df.columns)
                    batch_size = df.batch_size.values.tolist()
                    converter = df.runtime.values.tolist()
                    original_load_time = df.original_load_time.values.tolist()
                    original_infererence_time = df.original_infererence_time.values.tolist()
                    #runtime_conversion_time = df.runtime_conversion_time.values.tolist()
                    #runtime_saving_time = df.runtime_saving_time.values.tolist()
                    runtime_load_time = df.runtime_load_time.values.tolist()
                    runtime_inference_time = df.runtime_inference_time.values.tolist()
                    encoded_miss_classified_original_runtime_percentage = df.encoded_miss_classified_original_runtime_percentage.values.tolist()
                    encoded_miss_classified_original_test_runtime_percentage = df.encoded_miss_classified_original_test_runtime_percentage.values.tolist()
                    
                    for j in range(len(converter)):
                        if batch_size[j] == 128:
                            data_original_load_time.append(original_load_time[j])
                            data_original_infererence_time.append(original_infererence_time[j])
                            if converter[j] in data_runtime_load_time.keys():
                                #data_inference_before[converter[j]].append(original_infererence_time[j])
                                #data_runtime_conversion_time[converter[j]].append(runtime_conversion_time[j])
                                data_runtime_load_time[converter[j]].append(runtime_load_time[j])
                                data_runtime_inference_time[converter[j]].append(runtime_inference_time[j])

                                #data_memory_before[converter[j]].append(original_size[j])
                                data_encoded_miss_classified_original_test_runtime_percentage[converter[j]] += float(str(encoded_miss_classified_original_runtime_percentage[j]).replace('%', ''))
                                data_encoded_miss_classified_original_runtime_percentage[converter[j]] += float(str(encoded_miss_classified_original_test_runtime_percentage[j]).replace('%', ''))
                            else:
                                #data_inference_before[converter[j]] = [original_infererence_time[j]]
                                #data_runtime_conversion_time[converter[j]] = [runtime_conversion_time[j]]
                                
                                data_runtime_load_time[converter[j]] = [runtime_load_time[j]]
                                data_runtime_inference_time[converter[j]] = [runtime_inference_time[j]]

                                #data_memory_before[converter[j]] = [original_size[j]]
                                data_encoded_miss_classified_original_test_runtime_percentage[converter[j]] = float(str(encoded_miss_classified_original_runtime_percentage[j]).replace('%', ''))
                                data_encoded_miss_classified_original_runtime_percentage[converter[j]] = float(str(encoded_miss_classified_original_test_runtime_percentage[j]).replace('%', ''))
            #for key, val in data_inference_before.items()
            total_len = len(data_runtime_load_time['onnx'])
            #data_writer.writerow([frame_,model, np.mean(data_inference_before), np.mean(data_inference_after['onnx']), np.mean(data_inference_after['coremltools']), np.mean(data_memory_before), np.mean(data_memory_after['onnx']), np.mean(data_memory_after['coremltools'])])
            data_writer.writerow([frame_,model, '{:.2e}'.format(np.mean(data_original_load_time)), '{:.2e}'.format(np.mean(data_runtime_load_time['onnx'])), '{:.2e}'.format(np.mean(data_runtime_load_time['coremltools'])), '{:.2f}%'.format(np.sum(data_encoded_miss_classified_original_runtime_percentage['onnx'])/(100*total_len)), '{:.2f}%'.format(np.sum(data_encoded_miss_classified_original_runtime_percentage['coremltools'])/(100*total_len)), '{:.2e}'.format(np.mean(original_infererence_time)), '{:.2e}'.format(np.mean(data_runtime_inference_time['onnx'])), '{:.2e}'.format(np.mean(data_runtime_inference_time['coremltools'])), '',
                                 '{:.2e}'.format(np.std(data_original_load_time)), '{:.2e}'.format(np.std(data_runtime_load_time['onnx'])), '{:.2e}'.format(np.std(data_runtime_load_time['coremltools'])), '{:.2f}%'.format(np.std(data_encoded_miss_classified_original_runtime_percentage['onnx'])), '{:.2f}%'.format(np.std(data_encoded_miss_classified_original_runtime_percentage['coremltools'])), '{:.2e}'.format(np.std(original_infererence_time)), '{:.2e}'.format(np.std(data_runtime_inference_time['onnx'])), '{:.2e}'.format(np.std(data_runtime_inference_time['coremltools'])), 
                                 '{:.2f}%'.format(np.sum(data_encoded_miss_classified_original_runtime_percentage['onnx'])/(100*total_len)), '{:.2f}%'.format(np.sum(data_encoded_miss_classified_original_runtime_percentage['coremltools'])/(100*total_len))])


data_file.close()

In [34]:
original_infererence_time

[2.5649452209472656,
 2.5649452209472656,
 2.65227484703064,
 2.65227484703064,
 2.712142944335937,
 2.712142944335937,
 2.676571846008301,
 2.676571846008301,
 2.410376071929932,
 2.410376071929932,
 2.440882921218872,
 2.440882921218872,
 2.424577236175537,
 2.424577236175537,
 2.788219928741455,
 2.788219928741455,
 2.84631896018982,
 2.84631896018982,
 2.8421571254730225,
 2.8421571254730225,
 2.8847968578338623,
 2.8847968578338623]

In [5]:
data_file = open(path_output+'metrics_miss-classifications2.csv', mode='w', newline='', encoding='utf-8')
data_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#data_writer.writerow(['model','Epoch', 'Training_accuracy', 'Validation_Accuracy', "Training_Loss", 'Validation_Loss'])
data_writer.writerow(['framework','model', 'loadtime_before', 'loadtime_onnx', 'loadtime_coreml', 'miss_classification_onnx', 'miss_classification_coreml', 'inferencetime_before', 'inferencetime_onnx', 'inferencetime_coreml', '', 'loadtime_before_std', 'loadtime_onnx_std', 'loadtime_coreml_std', 'miss_classification_onnx_std', 'miss_classification_coreml_std', 'inferencetime_before_std', 'inferencetime_onnx_std', 'inferencetime_coreml_std', 'miss_test_onnx', 'miss_test_coreml'])

for frame_ in frameworks:
    for model in models: 
        data_original_load_time_2 = []
        data_original_infererence_time_2 = []
        data_runtime_conversion_time_2 = {}
        data_runtime_load_time_2 = {}
        data_runtime_inference_time_2 = {}
        data_encoded_miss_classified_original_test_runtime_percentage_2 = {}
        data_encoded_miss_classified_original_runtime_percentage_2 = {}

        if os.path.exists(path+'errors/{}/{}'.format(frame_,model)):
            list_files = [x for x in os.listdir(path+'errors/{}/{}'.format(frame_,model)) if 'runtime_miss-classification' in x]
            #print('yes', list_files)
            for i in range(1, 11):
                for file_ in list_files:
                    df = pd.read_csv(path+'errors/{}/{}/{}'.format(frame_,model, file_))
                    #print(df.columns)
                    batch_size = df.batch_size.values.tolist()
                    converter = df.runtime.values.tolist()
                    original_load_time = df.original_load_time.values.tolist()
                    original_infererence_time = df.original_infererence_time.values.tolist()
                    #runtime_conversion_time = df.runtime_conversion_time.values.tolist()
                    #runtime_saving_time = df.runtime_saving_time.values.tolist()
                    runtime_load_time = df.runtime_load_time.values.tolist()
                    runtime_inference_time = df.runtime_inference_time.values.tolist()
                    encoded_miss_classified_original_runtime_percentage = df.encoded_miss_classified_original_runtime_percentage.values.tolist()
                    encoded_miss_classified_original_test_runtime_percentage = df.encoded_miss_classified_original_test_runtime_percentage.values.tolist()
                    
                    data_original_load_time = []
                    data_original_infererence_time = []
                    data_runtime_conversion_time = {}
                    data_runtime_load_time = {}
                    data_runtime_inference_time = {}
                    data_encoded_miss_classified_original_test_runtime_percentage = {}
                    data_encoded_miss_classified_original_runtime_percentage = {}
        
                    for j in range(len(converter)):
                        if batch_size[j] == 128:
                            data_original_load_time.append(original_load_time[j])
                            data_original_infererence_time.append(original_infererence_time[j])
                            if converter[j] in data_runtime_load_time.keys():
                                #data_inference_before[converter[j]].append(original_infererence_time[j])
                                #data_runtime_conversion_time[converter[j]].append(runtime_conversion_time[j])
                                data_runtime_load_time[converter[j]].append(runtime_load_time[j])
                                data_runtime_inference_time[converter[j]].append(runtime_inference_time[j])

                                #data_memory_before[converter[j]].append(original_size[j])
                                data_encoded_miss_classified_original_test_runtime_percentage[converter[j]] += float(str(encoded_miss_classified_original_runtime_percentage[j]).replace('%', ''))
                                data_encoded_miss_classified_original_runtime_percentage[converter[j]] += float(str(encoded_miss_classified_original_test_runtime_percentage[j]).replace('%', ''))
                            else:
                                #data_inference_before[converter[j]] = [original_infererence_time[j]]
                                #data_runtime_conversion_time[converter[j]] = [runtime_conversion_time[j]]
                                
                                data_runtime_load_time[converter[j]] = [runtime_load_time[j]]
                                data_runtime_inference_time[converter[j]] = [runtime_inference_time[j]]

                                #data_memory_before[converter[j]] = [original_size[j]]
                                data_encoded_miss_classified_original_test_runtime_percentage[converter[j]] = float(str(encoded_miss_classified_original_runtime_percentage[j]).replace('%', ''))
                                data_encoded_miss_classified_original_runtime_percentage[converter[j]] = float(str(encoded_miss_classified_original_test_runtime_percentage[j]).replace('%', ''))
                
                    keys_ = ['onnx', 'coremltools']
                    for key_ in keys_:
                        if key_ in data_runtime_load_time_2.keys():
                            len_ = data_runtime_load_time[key_]
                            #print(data_runtime_load_time)
                            data_runtime_load_time_2[key_].append(np.mean(data_runtime_load_time[key_]))
                            data_runtime_inference_time_2[key_].append(np.mean(data_runtime_inference_time[key_]))
                            #data_original_load_time_2[key_].append(np.mean(data_runtime_load_time[key_]))
                            data_encoded_miss_classified_original_test_runtime_percentage_2[key_].append(np.sum(data_encoded_miss_classified_original_test_runtime_percentage[key_])/len_)
                            data_encoded_miss_classified_original_runtime_percentage_2[key_].append(np.sum(data_encoded_miss_classified_original_runtime_percentage[key_])/len_)
                            #data_original_load_time_2.append(np.mean(data_original_load_time))
                            #data_original_infererence_time_2.append(np.mean(data_original_infererence_time))
                        else:
                            len_ = data_runtime_load_time[key_]
                            data_runtime_load_time_2[key_] = [np.mean(data_runtime_load_time[key_])]
                            data_runtime_inference_time_2[key_] = [np.mean(data_runtime_inference_time[key_])]
                            #data_original_load_time_2[key_].append(np.mean(data_runtime_load_time[key_]))
                            data_encoded_miss_classified_original_test_runtime_percentage_2[key_] = [np.sum(data_encoded_miss_classified_original_test_runtime_percentage[key_])/len_]
                            data_encoded_miss_classified_original_runtime_percentage_2[key_] = [np.sum(data_encoded_miss_classified_original_runtime_percentage[key_])/len_]

                    
            #for key, val in data_inference_before.items()
            total_len = len(data_runtime_load_time['onnx'])
            #data_writer.writerow([frame_,model, np.mean(data_inference_before), np.mean(data_inference_after['onnx']), np.mean(data_inference_after['coremltools']), np.mean(data_memory_before), np.mean(data_memory_after['onnx']), np.mean(data_memory_after['coremltools'])])
            data_writer.writerow([frame_,model, '{:.2e}'.format(np.mean(data_original_load_time_2)), '{:.2e}'.format(np.mean(data_runtime_load_time_2['onnx'])), '{:.2e}'.format(np.mean(data_runtime_load_time_2['coremltools'])), '{:.2f}%'.format(np.mean(data_encoded_miss_classified_original_runtime_percentage_2['onnx'])), '{:.2f}%'.format(np.mean(data_encoded_miss_classified_original_runtime_percentage_2['coremltools'])), '{:.2e}'.format(np.mean(data_original_infererence_time_2)), '{:.2e}'.format(np.mean(data_runtime_inference_time_2['onnx'])), '{:.2e}'.format(np.mean(data_runtime_inference_time_2['coremltools'])), '',
                                 '{:.2e}'.format(np.std(data_original_load_time_2)), '{:.2e}'.format(np.std(data_runtime_load_time_2['onnx'])), '{:.2e}'.format(np.std(data_runtime_load_time_2['coremltools'])), '{:.2f}%'.format(np.std(data_encoded_miss_classified_original_runtime_percentage_2['onnx'])), '{:.2f}%'.format(np.std(data_encoded_miss_classified_original_runtime_percentage_2['coremltools'])), '{:.2e}'.format(np.std(original_infererence_time)), '{:.2e}'.format(np.std(data_runtime_inference_time['onnx'])), '{:.2e}'.format(np.std(data_runtime_inference_time['coremltools'])), 
                                 '{:.2f}%'.format(np.mean(data_encoded_miss_classified_original_runtime_percentage_2['onnx'])), '{:.2f}%'.format(np.mean(data_encoded_miss_classified_original_runtime_percentage_2['coremltools']))])


data_file.close()

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nda

ValueError: operands could not be broadcast together with shapes (23,) (25,) 